In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import dgl
import dgl.function as fn
import numpy as np
from sklearn.metrics import roc_auc_score

# define the R-GCN model
class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, num_rels, num_bases):
        super(RGCN, self).__init__()
        self.in_feats = in_feats
        self.hid_feats = hid_feats
        self.out_feats = out_feats
        self.num_rels = num_rels
        self.num_bases = num_bases

        # define the weight matrices for each relation
        self.w = nn.ModuleList()
        for i in range(num_rels):
            if num_bases > 0:
                self.w.append(nn.Linear(in_feats, num_bases))
            else:
                self.w.append(nn.Linear(in_feats, hid_feats))
        self.v = nn.Linear(hid_feats * num_rels, out_feats)

    def forward(self, g, h, r, norm):
        # compute the embeddings for each relation
        h = h.float()
        weight_list = []
        for i in range(self.num_rels):
            if self.num_bases > 0:
                weight = self.w[i](h).view(-1, self.hid_feats, self.num_bases)
                weight = torch.matmul(r[i], weight).view(-1, self.hid_feats * self.num_bases)
            else:
                weight = torch.matmul(r[i], self.w[i](h))
            weight_list.append(weight)

        # sum the embeddings over all relations
        h = torch.cat(weight_list, dim=1)
        h = self.v(h)

        # normalize the embeddings
        h = h * norm
        return h

# set the hyperparameters
in_feats = 10
hid_feats = 16
out_feats = 1
num_rels = 2
num_bases = 2
lr = 0.01
n_epochs = 50

# generate a random graph with 100 nodes and 1000 edges
g = dgl.rand_graph(100, 1000)
src, dst = g.edges()
labels = g.adjacency_matrix().to_dense()[src, dst]

# generate random features for each node
h = torch.randn(g.num_nodes(), in_feats)

# compute the normalization factor for each node
deg = g.in_degrees().float().clamp(min=1)
norm = torch.pow(deg, -0.5)

# split the data into training and testing sets
train_mask = torch.zeros(g.num_edges(), dtype=torch.bool)
test_mask = torch.zeros(g.num_edges(), dtype=torch.bool)
train_mask[:800] = True
test_mask[800:] = True
train_idx = torch.nonzero(train_mask).squeeze()
test_idx = torch.nonzero(test_mask).squeeze()

# create the R-GCN model and define the loss function and optimizer
model = RGCN(in_feats, hid_feats, out_feats, num_rels, num_bases)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# train the model
train_loss_list = []
for epoch in range(n_epochs):
    # set the model to train mode
    model.train()

    # zero the gradients
    optimizer.zero_grad()

    # forward pass
    logits = model(g, h, g.edata['type'], norm)
    loss = criterion(logits[train_idx], labels[train_idx].float())

    #
    # backward pass
    loss.backward()

    # update the parameters
    optimizer.step()

    # evaluate the model on the test set
    with torch.no_grad():
        # set the model to evaluation mode
        model.eval()

        # forward pass
        logits = model(g, h, g.edata['type'], norm)

        # compute the loss and accuracy
        test_loss = criterion(logits[test_idx], labels[test_idx].float())
        test_pred = torch.sigmoid(logits[test_idx])
        test_auc = roc_auc_score(labels[test_idx].numpy(), test_pred.numpy())

        # append the loss to the list
        train_loss_list.append(loss.item())

        # print the progress
        print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}, Test AUC: {:.4f}'.format(epoch+1, n_epochs, loss.item(), test_loss.item(), test_auc))

# plot the training loss over time
import matplotlib.pyplot as plt
plt.plot(range(1, n_epochs+1), train_loss_list)
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.show()
